# Crunching data

## Library import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading the data

In [2]:
path = "./data/"
cities_csv = "cities.csv"
providers_csv ="providers.csv"
stations_csv ="stations.csv"
ticket_data_csv ="ticket_data.csv"

In [3]:
read_file = lambda p,f : pd.read_csv(p+f, sep=",",on_bad_lines='warn', encoding="utf8")
cities_df = read_file(path,cities_csv) 
providers_df = read_file(path,providers_csv)  
stations_df = read_file(path,stations_csv)  
ticket_data_df = read_file(path,ticket_data_csv)  

## First sight at the file, check of the separators and encoding

In [4]:
print(" Cities column names",list(cities_df.columns), "\n --------------------------")
print(" Providers column names",list(providers_df.columns), "\n --------------------------")
print(" Stations column names",list(stations_df.columns), "\n --------------------------")
print(" Ticket data column names",list(ticket_data_df.columns), "\n --------------------------")

 Cities column names ['id', 'local_name', 'unique_name', 'latitude', 'longitude', 'population'] 
 --------------------------
 Providers column names ['id', 'company_id', 'provider_id', 'name', 'fullname', 'has_wifi', 'has_plug', 'has_adjustable_seats', 'has_bicycle', 'transport_type'] 
 --------------------------
 Stations column names ['id', 'unique_name', 'latitude', 'longitude'] 
 --------------------------
 Ticket data column names ['id', 'company', 'o_station', 'd_station', 'departure_ts', 'arrival_ts', 'price_in_cents', 'search_ts', 'middle_stations', 'other_companies', 'o_city', 'd_city'] 
 --------------------------


In [5]:
cities_df.head(8)

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN
5,10314,"Град Пожаревац, Централна Србија, Србија",град-пожаревац,44.619095,21.176522,NaN
6,11155,"Bussy-Lettrée, Grand-Est, France",bussy-lettree,48.804600,4.259500,NaN
7,11788,"Chamonix, Valle d'Aosta, Italia",chamonix-italia,45.817156,6.952375,NaN


We see that special characters are well encoded so we can keep utf8 as encoding

In [6]:
providers_df[providers_df.fullname.str.contains("é")]

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
215,8377,10,2,corailintercite,Intercités,False,False,True,False,train
219,8381,10,6,coraillunea,Intercités de Nuit,False,False,True,False,train
221,8383,10,9,corail,Intercités,False,False,True,False,train


Same for providers.csv

In [7]:
ticket_data_df.dtypes

id                   int64
company              int64
o_station          float64
d_station          float64
departure_ts        object
arrival_ts          object
price_in_cents       int64
search_ts           object
middle_stations     object
other_companies     object
o_city               int64
d_city               int64
dtype: object

In [8]:
ticket_data_df.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


ticket_data are numerics value or dates

In [9]:
[print(stations_df[stations_df.unique_name.str.contains(s)].head(1)) for s in ["Č","é","ç"]]


      id          unique_name   latitude  longitude
136  137  Čakovec bus station  46.391354  16.434993
   id                 unique_name   latitude  longitude
1   2  Aéroport Bordeaux-Mérignac  44.830226  -0.700883
      id                      unique_name   latitude  longitude
797  798  Montluçon (Central bus station)  46.337443   2.597606


[None, None, None]

Here again, the special character are well printed so the encoding is good for all the files. We can now start to clean the data if necessary

We see that the columns o_city and d_city in the dataframe ticket_data are foreign key to the column id in cities_df.

The column company_id in providers_df is secondary key, id is the primary key and provider_id can be null so maybe it's an inner key to link two providers. It may not be useful so I keep it to later. company_id is use to link a ticket to a company by the column company and other_companies in tickey_data_df.

stations_df has the column id as primary key and is linked to the columns o_station, d_stations and middle_stations in ticket_data_df.

## Cleaning the data

In [10]:
cities_df[cities_df.id.isna()]

,id,local_name,unique_name,latitude,longitude,population


In [11]:
cities_df.count()

id             8040
local_name     8040
unique_name    8039
latitude       8040
longitude      8040
population      369
dtype: int64

Many of population data are NaN so this column maybe not usable and unique_name has one NaN value

In [12]:
cities_df.head()

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [13]:
cities_df["unique_name"]=cities_df.unique_name.str.lower()

In [14]:
cities_df[ cities_df.unique_name.isna()]

,id,local_name,unique_name,latitude,longitude,population
6866,10244,", Hamburg, Deutschland",NaN,39.613402,2.880431,NaN


This data is odd, the latitude and longitude doesn't fit with Deutshland but with Balearic Catalan island

In [15]:
cities_df[cities_df.id==10244]

,id,local_name,unique_name,latitude,longitude,population
6866,10244,", Hamburg, Deutschland",NaN,39.613402,2.880431,NaN


In [16]:
ticket_data_df[ (ticket_data_df.o_city==10244) | (ticket_data_df.d_city==10244)] 

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city


We observe that the city with id 10244 isn't used in the tickets' data so we can delete it without consequence

In [17]:
cities_df=cities_df[cities_df.id!=10244]

In [18]:
cities_df[cities_df.id.duplicated()]

,id,local_name,unique_name,latitude,longitude,population


In [19]:
cities_df[cities_df.unique_name.duplicated()]

,id,local_name,unique_name,latitude,longitude,population


Cities data haven't duplicate id nor unique_name so the data are clean for this two features

In [20]:
cities_df[ cities_df.unique_name.str.islower()]

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN
...,...,...,...,...,...,...
8035,9964,"Saint-André-lez-Lille, Hauts-de-France, France",saint-andre-lez-lille,50.654250,3.047690,NaN
8036,10863,"Kiev, Шевченківський район, Ukraine",kiev,50.450100,30.523400,NaN
8037,11144,"Brighton, England, UK",brighton,50.822000,-0.137400,NaN
8038,11377,"Manises, Comunitat Valenciana, España",manises,39.493000,-0.462100,NaN


We have one unique_name value that is not empty nor lower case which implies some special charaters

In [21]:
cities_df[cities_df.local_name.str.contains(", ,")]

,id,local_name,unique_name,latitude,longitude,population
49,10711,", ,",-,1.370676,48.731938,NaN
69,9393,"Ampolla Perello, Cataluña, , España",ampolla-perello,40.814300,0.714600,NaN
700,11701,"Минск, , Беларусь",минск,53.893000,28.032700,NaN
2429,7687,"MINSK-PASSAJIRSKII., , Беларусь",minsk-passajirskii.,53.890700,27.550600,NaN
7701,11606,", , United Kingdom",-united kingdom,51.484900,-0.124200,NaN
8022,7711,"Monaco - Monte-Carlo, , Monaco - Monte-Carlo",monaco---monte-carlo,43.738740,7.419400,NaN


We observe that the rows of id 10711 and 11606 are odd. As before we look if the table ticket_data are linked to these data

In [22]:
ticket_data_df[ (ticket_data_df.o_city.isin([10711,11606]) ) | (ticket_data_df.d_city.isin([10711,11606]))] 

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city


These data is not used so we can delete them

In [23]:
cities_df=cities_df[~cities_df.id.isin([10711,11606])]

We observe that local_name is not an atomic column so we can divide it in three column: city, region and country

In [24]:
def get_pod_comma(s:str):
    result=[]
    [(result.append(i) if s[i]==',' else None) for i in range(len(s))]
    if len(result)<2: result=[0,0]
    return result

In [25]:
cities_df.loc[:, "city"] = cities_df["local_name"].apply(lambda x: x.split(',')[0].strip())
cities_df.loc[:, "state"] = cities_df["local_name"].apply(lambda x: x[x.find(',') + 1 : get_pod_comma(x)[1]].strip())
cities_df.loc[:, "country"] = cities_df["local_name"].apply(lambda x: x[get_pod_comma(x)[1]+1:].strip())

In [26]:
cities_df.loc[:, ("id","local_name","city","state", "country")].head()

,id,local_name,city,state,country
0,5159,"Padua, Veneto, Italia",Padua,Veneto,Italia
1,76,"Barcelona, Cataluña, España",Barcelona,Cataluña,España
2,81,"Basel, Basel-Stadt, Schweiz",Basel,Basel-Stadt,Schweiz
3,259,"Erlangen, Bayern, Deutschland",Erlangen,Bayern,Deutschland
4,11979,"Balș, Olt, România",Balș,Olt,România


In [27]:
cities_df.count()

id             8037
local_name     8037
unique_name    8037
latitude       8037
longitude      8037
population      369
city           8037
state          8037
country        8037
dtype: int64

In [28]:
[print(cities_df[s].isna().count()) for s in ["city","state","country"]]

8037
8037
8037


[None, None, None]

The new columns haven't na values so all seem clean for this data.

In [29]:
[print(cities_df[cities_df[s].str.contains(",")].empty) for s in ["city","state","country"] ]

True
True
False


[None, None, None]

In [30]:
cities_df[cities_df["state"].str.contains("Grand-Est")]

,id,local_name,unique_name,latitude,longitude,population,city,state,country
6,11155,"Bussy-Lettrée, Grand-Est, France",bussy-lettree,48.804600,4.259500,NaN,Bussy-Lettrée,Grand-Est,France
14,12005,"Gare de Lamarche, Grand-Est, France",gare-de-lamarche,48.089980,5.760170,NaN,Gare de Lamarche,Grand-Est,France
24,12016,"Hymont, Grand-Est, France",hymont,48.265700,6.141900,NaN,Hymont,Grand-Est,France
29,12023,"Jessains, Grand-Est, France",jessains,48.297070,4.577940,NaN,Jessains,Grand-Est,France
30,12029,"Knutange, Grand-Est, France",knutange,49.340510,6.038290,NaN,Knutange,Grand-Est,France
...,...,...,...,...,...,...,...,...,...
7937,12034,"Lac d'Orient (Mesnil-Saint-Père), Grand-Est, F...",lac-dorient-,48.248977,4.333515,NaN,Lac d'Orient (Mesnil-Saint-Père),Grand-Est,France
7946,12020,"Ingersheim, Grand-Est, France",ingersheim,48.088940,7.297690,NaN,Ingersheim,Grand-Est,France
7958,12065,"Nigloland (Dolancourt), Grand-Est, France",nigloland-,48.265910,4.616850,NaN,Nigloland (Dolancourt),Grand-Est,France
7975,12118,"Bâle Mulhouse Fribourg Aéroport, Grand-Est, Fr...",bale-mulhouse-fribourg-aeroport,47.600746,7.531787,NaN,Bâle Mulhouse Fribourg Aéroport,Grand-Est,France


In [31]:
cities_df[cities_df["state"]==""]

,id,local_name,unique_name,latitude,longitude,population,city,state,country
25,12017,"Ile de Ré Nouvelle-Aquitaine, France",ile-de-re-nouvelle-aquitaine,46.228940,-1.541300,NaN,Ile de Ré Nouvelle-Aquitaine,,"le de Ré Nouvelle-Aquitaine, France"
99,1418,"Branksome, EnglandUnited Kingdom",branksome,50.722411,-1.927281,NaN,Branksome,,"ranksome, EnglandUnited Kingdom"
407,8260,"Recco, Liguria Italia",recco,44.361100,9.146700,NaN,Recco,,"ecco, Liguria Italia"
600,5199,WTF OMG,wtf-om,40.929715,14.528693,NaN,WTF OMG,,TF OMG
700,11701,"Минск, , Беларусь",минск,53.893000,28.032700,NaN,Минск,,Беларусь
1421,9926,Regourd-Cahors,regourd-cahor,44.470540,1.427600,NaN,Regourd-Cahors,,egourd-Cahors
1735,5559,"Arosa, Schweiz",arosa,46.780000,9.678600,NaN,Arosa,,"rosa, Schweiz"
2027,9447,Bitterfeld,bitterfel,51.623300,12.315600,NaN,Bitterfeld,,itterfeld
2067,9494,"Catania, Sicilia Italia",catania,37.506100,15.099400,NaN,Catania,,"atania, Sicilia Italia"
2429,7687,"MINSK-PASSAJIRSKII., , Беларусь",minsk-passajirskii.,53.890700,27.550600,NaN,MINSK-PASSAJIRSKII.,,Беларусь


In [32]:
ticket_data_df[ (ticket_data_df.o_city==5199 ) | (ticket_data_df.d_city==5199)] 

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city


"WTF OMG" with city id 5199 isn't used in the ticket_station so we can remove it

In [33]:
cities_df=cities_df[cities_df.id!=5199]

15 rows have the state column empty, sometimes du to bad format of unique_name column. We try to correct them:

In [34]:
cities_df.loc[(cities_df.id==12017), ["city"]]="Ile de Ré"
cities_df.loc[(cities_df.id==2517), ["city"]]="Svilajnac"

cities_df.loc[(cities_df.id==12017), ["state"]]="Nouvelle-Aquitaine"
cities_df.loc[(cities_df.id==1418), ["state"]]="England"
cities_df.loc[(cities_df.id==8260), ["state"]]="Liguria"
cities_df.loc[(cities_df.id==9494), ["state"]]="Sicilia "
cities_df.loc[(cities_df.id==9926), ["state"]]="Occitanie"
cities_df.loc[(cities_df.id==5559), ["state"]]="Graubünden"
cities_df.loc[(cities_df.id==9447), ["state"]]="Sachsen-Anhalt"
cities_df.loc[(cities_df.id==9719), ["state"]]="Bretagne"
cities_df.loc[(cities_df.id==10245), ["state"]]="Ivano-Frankivsk"
cities_df.loc[(cities_df.id==2517), ["state"]]="Central Serbia"
cities_df.loc[(cities_df.id==5406), ["state"]]="Grad Zagreb"
cities_df.loc[(cities_df.id==8171), ["state"]]="Cataluña"
cities_df.loc[(cities_df.id==9680), ["state"]]="Nordrhein-Westfalen"
cities_df.loc[(cities_df.id==9681), ["state"]]="Grand-Est"

cities_df.loc[(cities_df.id==12017), ["country"]]="France"
cities_df.loc[(cities_df.id==1418), ["country"]]="United Kingdom"
cities_df.loc[(cities_df.id==8260), ["country"]]="Italia"
cities_df.loc[(cities_df.id==9494), ["country"]]="Italia"
cities_df.loc[(cities_df.id==9926), ["country"]]="France"
cities_df.loc[(cities_df.id==5559), ["country"]]="Schweiz"
cities_df.loc[(cities_df.id==9447), ["country"]]="Deutschland"
cities_df.loc[(cities_df.id==10245), ["country"]]="Ukraine"
cities_df.loc[(cities_df.id==9719), ["country"]]="France"
cities_df.loc[(cities_df.id==2517), ["country"]]="Serbia"
cities_df.loc[(cities_df.id==5406), ["country"]]="Hrvatska"
cities_df.loc[(cities_df.id==8171), ["country"]]="España"
cities_df.loc[(cities_df.id==9680), ["country"]]="Deutschland"
cities_df.loc[(cities_df.id==9681), ["country"]]="France"

In [35]:
print(cities_df.loc[(cities_df.country.str.contains("United Kingdom"))].country.count())
print(cities_df.loc[(cities_df.country.str.contains("UK"))].country.count())

282
230


In [36]:
cities_df.loc[(cities_df.country.str.contains("United Kingdom"))].head()

,id,local_name,unique_name,latitude,longitude,population,city,state,country
82,1432,"Bristol Emersons Green, England, United Kingdom",bristol-emersons-green,51.494847,-2.445600,NaN,Bristol Emersons Green,England,United Kingdom
99,1418,"Branksome, EnglandUnited Kingdom",branksome,50.722411,-1.927281,NaN,Branksome,England,United Kingdom
101,1430,"Brierfield, England, United Kingdom",brierfield,53.878509,-2.171497,NaN,Brierfield,England,United Kingdom
121,1754,"Highbridge, England, United Kingdom",highbridge,51.186834,-2.968421,NaN,Highbridge,England,United Kingdom
139,2063,"Mirfield, England, United Kingdom",mirfield,53.642780,-1.780614,NaN,Mirfield,England,United Kingdom


In [37]:
cities_df.loc[(cities_df.country.str.contains("UK"))]

,id,local_name,unique_name,latitude,longitude,population,city,state,country
389,10261,"Lamorick, England, UK",lamorick,50.448305,-4.768291,NaN,Lamorick,England,UK
656,10377,"St Albans, England, UK",st-albans,51.749296,-0.342253,NaN,St Albans,England,UK
709,1705,"Great Barr, England, UK",great-barr,52.559357,-1.930136,NaN,Great Barr,England,UK
880,10335,"Vauxhall, England, UK",vauxhall,52.485467,-1.875952,NaN,Vauxhall,England,UK
1030,2551,"Theale, England, UK",theale,51.438064,-1.076722,NaN,Theale,England,UK
...,...,...,...,...,...,...,...,...,...
8028,2126,"Norwich, England, UK",norwich,52.628606,1.292270,186682.0,Norwich,England,UK
8030,2249,"Portsmouth, England, UK",portsmouth,50.803683,-1.075614,238137.0,Portsmouth,England,UK
8031,2448,"Southampton, England, UK",southampton,50.902535,-1.404189,253651.0,Southampton,England,UK
8037,11144,"Brighton, England, UK",brighton,50.822000,-0.137400,NaN,Brighton,England,UK


For most of the rows about United kingdom, we see that the state is both UK or United Kindom, we need to uniformize it. 

In [38]:
cities_df.loc[(cities_df.country.str.contains("UK")), ["country"]]="United Kingdom"

In [39]:
cities_df.loc[cities_df["country"].str.contains(","),("id","local_name","city","state", "country")]

,id,local_name,city,state,country
69,9393,"Ampolla Perello, Cataluña, , España",Ampolla Perello,Cataluña,", España"
327,10440,"Gaillard - Suisse, Genève, Schweiz, Suisse, Sv...",Gaillard - Suisse,Genève,"Schweiz, Suisse, Svizzera, Svizra"
334,10608,"Horgen, Zürich, Schweiz, Suisse, Svizzera, Svizra",Horgen,Zürich,"Schweiz, Suisse, Svizzera, Svizra"
373,1254,"Venezia MESTRE, Venezia MESTRE, Venezia, VEN, ...",Venezia MESTRE,Venezia MESTRE,"Venezia, VEN, VE, Italia"
395,10643,"Mendrisio, Ticino, Schweiz, Suisse, Svizzera, ...",Mendrisio,Ticino,"Schweiz, Suisse, Svizzera, Svizra"
...,...,...,...,...,...
7771,11922,"Nossa Senhora da Vila, Nossa Senhora do Bispo ...",Nossa Senhora da Vila,Nossa Senhora do Bispo e Silveiras,"Alentejo, Portugal"
7778,11519,"Sempach, Luzern, Schweiz, Suisse, Svizzera, Sv...",Sempach,Luzern,"Schweiz, Suisse, Svizzera, Svizra"
7817,11638,"Wil (SG), Sankt Gallen, Schweiz, Suisse, Svizz...",Wil (SG),Sankt Gallen,"Schweiz, Suisse, Svizzera, Svizra"
7885,11178,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",Cedofeita,Santo Ildefonso,"Sé, Miragaia, São Nicolau e Vitória, Norte, Po..."


We need to uniformize the name of Switzerland. We keep the Scheiz name.

In [40]:
cities_df.loc[(cities_df.country.isin(["Schweiz", "Suisse", "Svizzera", "Svizra"])), ("id","local_name","city","state", "country")].groupby("country").count()

,id,local_name,city,state
country,,,,
Schweiz,119,119,119,119


In [41]:
cities_df.loc[(cities_df.country=="Schweiz, Suisse, Svizzera, Svizra"),("country")]="Schweiz"

In [42]:
cities_df.loc[(cities_df.country=="Svizra"), ("country")]="Schweiz"

In [43]:
print(cities_df.loc[cities_df["country"].str.contains(","),("id")].count())
cities_df.loc[cities_df["country"].str.contains(","),("id","local_name","city","state", "country")]

25


,id,local_name,city,state,country
69,9393,"Ampolla Perello, Cataluña, , España",Ampolla Perello,Cataluña,", España"
373,1254,"Venezia MESTRE, Venezia MESTRE, Venezia, VEN, ...",Venezia MESTRE,Venezia MESTRE,"Venezia, VEN, VE, Italia"
555,1081,"Frankfurt am Main, Frankfurt am Main, Frankfur...",Frankfurt am Main,Frankfurt am Main,"Frankfurt, Hessen, Regierungsbezirk Darmstadt,..."
670,10789,"Coimbra, Santa Cruz, Almedina e São Bartolomeu...",Coimbra,Santa Cruz,"Almedina e São Bartolomeu, Centro, Portugal"
851,1060,"Brussel, Brussel, Schaerbeek - Schaarbeek, Rég...",Brussel,Brussel,"Schaerbeek - Schaarbeek, Région de Bruxelles-C..."
1075,1056,"Besse-et-Saint-Anastaise (Super Besse), Besse-...",Besse-et-Saint-Anastaise (Super Besse),Besse-et-Saint-Anastaise (Super Besse),"Besse-et-Saint-Anastaise, Auvergne-Rhône-Alpes..."
1583,1110,"Huez (Alpe d&#039;Huez), Huez (Alpe d&#039;Hue...",Huez (Alpe d&#039;Huez),Huez (Alpe d&#039;Huez),"Huez, Auvergne-Rhône-Alpes, Grenoble, France"
1969,1109,"Hranice na M., Hranice na M., Hranice, Střední...",Hranice na M.,Hranice na M.,"Hranice, Střední Morava, okres Přerov, Česko"
1970,1246,"Ústí n.L., Ústí n.L., Ústí nad Labem, Severozá...",Ústí n.L.,Ústí n.L.,"Ústí nad Labem, Severozápad, okres Ústí nad La..."
2244,1120,"Köln/Bonn Flughafen, Köln/Bonn Flughafen, Köln...",Köln/Bonn Flughafen,Köln/Bonn Flughafen,"Köln, Nordrhein-Westfalen, Regierungsbezirk Kö..."


They still persist some error on rows having a city name containing. We correct name manually

In [44]:
cities_df.loc[cities_df.id==9393	,("city")]="Ampolla Perello"
cities_df.loc[cities_df.id==1254	,("city")]="Venezia MESTRE"
cities_df.loc[cities_df.id==1081	,("city")]="Frankfurt am Main"
cities_df.loc[cities_df.id==10789	,("city")]="Coimbra, Santa Cruz, Almedina e São Bartolomeu"					
cities_df.loc[cities_df.id==1060	,("city")]="Brussel"														
cities_df.loc[cities_df.id==1056	,("city")]="Besse-et-Saint-Anastaise (Super Besse)"							
cities_df.loc[cities_df.id==1110	,("city")]="Huez (Alpe d'Huez)"												
cities_df.loc[cities_df.id==1109	,("city")]="Hranice na M."													
cities_df.loc[cities_df.id==1246	,("city")]="Ústí nad Labem"													
cities_df.loc[cities_df.id==1120	,("city")]="Köln/Bonn Flughafen"											
cities_df.loc[cities_df.id==2127	,("city")]="Nossa Senhora do Pópulo"										
cities_df.loc[cities_df.id==1239	,("city")]="Třinec centrum"													
cities_df.loc[cities_df.id==1077	,("city")]="Dvůr Králové nad Labem"											
cities_df.loc[cities_df.id==2403	,("city")]="Santa Maria e Meixedo"											
cities_df.loc[cities_df.id==830	,("city")]="Terlano, Trentro"													
cities_df.loc[cities_df.id==2402	,("city")]="Sé Nova, Santa Cruz, Almedina e São Bartolomeu"					
cities_df.loc[cities_df.id==2373	,("city")]="Santa Maria da Feira, Travanca, Sanfins e Espargo"				
cities_df.loc[cities_df.id==11508	,("city")]="Santa Maria, São Pedro e Matacães"								
cities_df.loc[cities_df.id==11511	,("city")]="São Gonçalo, Cepelos e Gatão"									
cities_df.loc[cities_df.id==10656	,("city")]="Perafita, Lavra e Santa Cruz do Bispo"							
cities_df.loc[cities_df.id==11611	,("city")]="Valença, Cristelo Covo e Arão"									
cities_df.loc[cities_df.id==11922	,("city")]="Nossa Senhora da Vila, Nossa Senhora do Bispo e Silveiras"		
cities_df.loc[cities_df.id==11178	,("city")]="Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória"
cities_df.loc[cities_df.id==1415	,("city")]="Braga, Sé e Cividade"											


In [45]:
cities_df.loc[cities_df.id==9393	,("state")]="Cataluña"
cities_df.loc[cities_df.id==1254	,("state")]="Venezia"
cities_df.loc[cities_df.id==1081	,("state")]="Hessen"
cities_df.loc[cities_df.id==10789	,("state")]="Centro"
cities_df.loc[cities_df.id==1060	,("state")]="Région de Bruxelles-Capitale"
cities_df.loc[cities_df.id==1056	,("state")]="Auvergne-Rhône-Alpes"
cities_df.loc[cities_df.id==1110	,("state")]="Auvergne-Rhône-Alpes"
cities_df.loc[cities_df.id==1109	,("state")]="Střední Morava"
cities_df.loc[cities_df.id==1246	,("state")]="Severozápad"
cities_df.loc[cities_df.id==1120	,("state")]="Nordrhein-Westfalen"
cities_df.loc[cities_df.id==2127	,("state")]="Centro"
cities_df.loc[cities_df.id==1239	,("state")]="Moravskoslezsko"
cities_df.loc[cities_df.id==1077	,("state")]="Severovýchod"
cities_df.loc[cities_df.id==2403	,("state")]="Norte"
cities_df.loc[cities_df.id==830	,("state")]="TerlTrentino-Alto Adige"
cities_df.loc[cities_df.id==2402	,("state")]="Centro"
cities_df.loc[cities_df.id==2373	,("state")]="Norte"
cities_df.loc[cities_df.id==11508	,("state")]="Centro"
cities_df.loc[cities_df.id==11511	,("state")]="Norte"
cities_df.loc[cities_df.id==10656	,("state")]="Norte"
cities_df.loc[cities_df.id==11611	,("state")]="Norte"
cities_df.loc[cities_df.id==11922	,("state")]="Alentejo"
cities_df.loc[cities_df.id==11178	,("state")]="Norte"
cities_df.loc[cities_df.id==1415	,("state")]="Norte"

We keep only the part after the last comma in the country column

In [46]:
cities_df.loc[(cities_df.country.str.contains(",")), ["country"]]=cities_df.loc[(cities_df.country.str.contains(",")), "local_name"].apply(lambda x: x.split(',')[-1].strip())

In [47]:
[print(s, "has null value") if not providers_df[providers_df[s].isna()].empty else None for s in list(providers_df.columns)]

provider_id has null value
has_wifi has null value
has_plug has null value
has_adjustable_seats has null value
has_bicycle has null value


[None, None, None, None, None, None, None, None, None, None]

In [48]:
providers_df[providers_df["has_wifi"].isna() | providers_df["has_plug"].isna() | 
            providers_df["has_adjustable_seats"].isna() | providers_df["has_bicycle"].isna() ]

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
212,8374,20,NaN,transdev,Transdev,NaN,NaN,NaN,NaN,bus
224,8387,30,NaN,vatry,Vatry,NaN,NaN,NaN,NaN,bus
225,8388,40,NaN,beauval,Beauval,NaN,NaN,NaN,NaN,bus


The column provider_id in provider_df has some nan as expected. Three rows have has_wifi, has_plug, has_adjustable_seats and has_bicycle with nan value. We decide to kept them to not loss data. As the data are about bus we choose to set them to false but this can be discussed

In [49]:
providers_df.loc[providers_df["has_wifi"].isna() | providers_df["has_plug"].isna() | 
            providers_df["has_adjustable_seats"].isna() | providers_df["has_bicycle"].isna() 
            ,["has_wifi","has_plug","has_adjustable_seats","has_bicycle"]]=[False,False,False,False]

In [50]:
[print(s, "has no duplicate", providers_df[providers_df[s].duplicated()].empty) for s in ["id", "name", "fullname"]]

id has no duplicate True
name has no duplicate True
fullname has no duplicate False


[None, None, None]

In [51]:
providers_df[providers_df[["fullname"]].duplicated()]

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
218,8380,10,5,train,TER,False,False,True,False,train
221,8383,10,9,corail,Intercités,False,False,True,False,train


In [52]:
providers_df[providers_df["fullname"].isin(["TER","Intercités"]) | providers_df.id.isin([0,2,5,9])]

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
213,8375,10,0,trainter,TER,False,False,True,False,train
215,8377,10,2,corailintercite,Intercités,False,False,True,False,train
218,8380,10,5,train,TER,False,False,True,False,train
221,8383,10,9,corail,Intercités,False,False,True,False,train


In [53]:
providers_df.loc[~providers_df["provider_id"].astype(str).str.isdigit(), "provider_id"]=np.nan

These data have duplicate full name but it doesn't seem to have many impact so I don't modify them

In [54]:
stations_df[stations_df["unique_name"].duplicated()]

,id,unique_name,latitude,longitude


In [55]:
stations_df[stations_df["id"].duplicated()]

,id,unique_name,latitude,longitude


In [56]:
ticket_data_df[ticket_data_df["id"].duplicated()]

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city


In [57]:
ticket_data_df["price_in_cents"].describe()

count    74168.000000
mean      4382.711061
std       3739.325367
min        300.000000
25%       1900.000000
50%       3350.000000
75%       5250.000000
max      38550.000000
Name: price_in_cents, dtype: float64

We transform the price to euro

In [58]:
ticket_data_df.rename(columns={"price_in_cents": "price_in_euros"}, inplace=True) 
ticket_data_df["price_in_euros"] = ticket_data_df["price_in_euros"] / 100.00

We also add a duration column

The data is now clean and can be exploited. They are maybe some discrepancy left but it will ask much more time and some more interesting task have to be done. We export the data in csv to reuse them in another notebook.

In [59]:
cities_df.to_csv("data/cities_cleaned.csv", sep=';', index= False)
stations_df.to_csv("data/stations_cleaned.csv", sep=';', index= False)
providers_df.to_csv("data/providers_cleaned.csv", sep=';', index= False)
ticket_data_df.to_csv("data/ticket_data_cleaned.csv", sep=';', index= False)